

### Imports



In [2]:
from PIL import Image, ImageFont, ImageDraw
import numpy
from pylab import *

import random
import os
import string
import json
import csv

from google.colab import drive
drive.mount('/content/drive')

### Fill the cards with data

In [ ]:

 # There should be 1 image in background folder 
bg_path = "/content/drive/MyDrive/p2-Data/backgrounds"  # images path
font_path = "/content/drive/MyDrive/p2-Data/fonts"   # Font path
output_path = "/content/drive/MyDrive/p2-Data/test/pakistan" # output file
num_images = 3000
show_bbox = False

font_color_pak = '#abc79b'
font_color_canada = '#6695bb'
font_color_china = '#c8d5df'
font_color_queensland = '#608814'
font_color_usa = '#68574d'

def idcard_draw(font, c_font, person, im_info) :
	#print("im_info = ", im_info)
	title_str = ' '
	pos = []
	# R, G, B = randint(10,245), randint(10,245), randint(10,245)
	margin_top = 10
	margin_down = 30
	
	color = (0,0,0) #randint(0,255),randint(0,255),randint(0,255))
	sz = font.getsize(title_str)
	x, y = (im_info[0]-sz[0])/2, margin_top
	draw.text((x,y), title_str, font_color_pak, font=font)
	
	Dy = (im_info[1]-margin_top-margin_down-sz[1]-30) / 4
	Dy = 50 #min(Dy, 30)
	y = margin_top + sz[1] + 110 #margins = 150 usa, 110 queensland,canada,pak, 200 china
	#print("y = {}, Dy = {}".format(y,Dy))
	#print("person = ", person)
	for key, val in person.items() :
		#print("key = ", key)
		#print("val = ", val)
		#print("y = ", y)
		x = 10
		sz_key = font.getsize(key)
		x = (im_info[0]-sz[0])/2-180
		#y = margin_top
		draw.text((x,y), key + ":", color, font=font)
		
		x += sz_key[0]+10
		str = val
		op = 11
		if op >= 10 :
			c = random.randint(0, len(str)-1)
			post = str[(c+1):]
			draw.text((x,y), str[:c], color, font=font)
			wpre, h = font.getsize(str[:c])
			if len(str) > c+1 :
				w, h = font.getsize(str)
				wpost, h = font.getsize(post)
				draw.text((x+w-wpost,y), str[c+1:], color, font=font)
				# r = R + randint(-10,10)
				# g = G + randint(-10,10)
				# b = B + randint(-10,10)
				# diam = randint(5,11)
				# draw.ellipse([x+w-wpost,y,x+w-wpost+diam,y+diam], fill=(r,g,b))
			dx = random.randint(-2,2)
			dy = random.randint(-2,2)
			while dx == 0 and dy == 0 :
				dx = random.randint(-2,2)
				dy = random.randint(-2,2)
			xc, yc = x+wpre+dx, y+dy
			draw.text((xc, yc), str[c], color, font=font)
			wc, hc = font.getsize(str[c])
			c_off = font.getoffset(str[c])
			if xc+c_off[0]>=0 and yc+c_off[1]>=0 and xc+wc<im_info[0] and yc+hc<im_info[1] :
				pos.append([xc+c_off[0], yc+c_off[1], xc+wc, yc+hc])
				if show_bbox :
					draw.rectangle(pos[-1], outline = 'blue')
		else :
			draw.text((x,y), str, color, font=font)
		y += Dy
			
	return pos

lists = []		
if __name__ == '__main__' :
	fontstyles = os.listdir(font_path)
	bgs = os.listdir(bg_path)
	pos_dict_list = []
	lists = pos_dict_list
	csv_file = open('/content/drive/MyDrive/p2-Data/DATA.csv','r')
	it = csv.DictReader(csv_file)
	persons = []
	for person in it :
		persons.append(person)
	
	
	for i in range(1,len(persons),1) : 
		if i >=num_images : 
			break
		if i % 1000 == 0 :
			print("i = {}".format(i))
   # Can change font style
		fontstyle = os.path.join(font_path, fontstyles[3]) 
		
		fontsize = 30 # random.randint(20,30)#
		font = ImageFont.truetype(fontstyle, fontsize)
		c_fontstyle = fontstyle
		c_fontsize = fontsize + random.randint(-2,2)
		c_font = ImageFont.truetype(c_fontstyle, c_fontsize)
		bg = os.path.join(bg_path, bgs[random.randint(0,len(bgs)-1)])
		print(bg)
		if not '.jpg' in bg:
			continue
		im = Image.open(bg)
		im_info = im.size
		print(im_info)
		draw = ImageDraw.Draw(im)
		
		pos = idcard_draw(font, c_font, dict(persons[i]), im_info)
		
		file_name = "card.{}.jpg".format(i)
		
		im.save(os.path.join(output_path, file_name),quality=95)
		
		pos_dict_list.append({'path':file_name, 'bbox':pos})

	with open(os.path.join(output_path, "annotation.json"),"w") as f :
		json.dump(pos_dict_list, f)


### Data Augmentation

In [ ]:
# rotation
def rotate_im(image, angle):
  
    # grab the dimensions of the image and then determine the
    # centre
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)

    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])

    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))

    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY

    # perform the actual rotation and return the image
    image = cv2.warpAffine(image, M, (nW, nH))

#    image = cv2.resize(image, (w,h))
    return image

def shear(img, shear_factor=0.1):
  shear_factor = random.uniform(shear_factor,0.2)

  w,h = img.shape[1], img.shape[0]


  M = np.array([[1, abs(shear_factor), 0],[0,1,0]])

  nW =  img.shape[1] + abs(shear_factor*img.shape[0])


  img = cv2.warpAffine(img, M, (int(nW), img.shape[0]))


  img = cv2.resize(img, (w,h))

  scale_factor_x = nW / w


  return img

### Randomly Tampering Tasks & Save it

In [ ]:
import cv2 as cv
import json 
import os
import numpy as np
import cv2



# Opening JSON file 
for j in ["Pakistan"]: #,"Canada","QueensLand","USA"
    path="/content/drive/MyDrive/p2-Data/test/pakistan"
    f = open(path+'/annotation.json',) 

    # returns JSON object as  
    # a dictionary 
    data = json.load(f) 

    # Iterating through the json 
    # list
    count= 0
    for img_name in os.listdir(path):
      
        if count == 1500:
          break
        print("Processing : ", j, "  ", count , "/ ", 1500)
        count+=1
        if not ".jpg" in img_name:
            continue
        im=cv.imread(path+"/"+img_name)
        ls = { i["path"]: i["bbox"]  for i  in data }
        f.close() 
        for i in ls[img_name]:
            x_min = int(i[1])
            y_min = int(i[0])
            x_max = int(i[3])
            y_max = int(i[2])
            im[ x_min:x_max, y_min:y_max]= im[ x_min:x_max, y_min:y_max]+np.random.randint(-100,100)  # Adding noise

            if count % 2 ==0:
              # rotation
              im = rotate_im(im, 5)
            elif count %2 !=0:
              # shearing
              im = shear(im)
            
            
            
        # Save randomly tampered card
        cv.imwrite(path+"/noised/"+img_name,im) 
    f.close()
    

